# Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [7]:
import sqlite3
import nltk
import random
import numpy as np
import re
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
nltk.download("punkt")
nltk.download("stopwords")

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

[nltk_data] Downloading package punkt to C:\Users\16302/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\16302/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
stop_words = set(stopwords.words("english"))

def clean_tokenize(text):
    tokens = word_tokenize(text.lower())
    return [word for word in tokens if word.isalpha() and word not in stop_words and word not in punctuation]

In [9]:
# Extract specific linguistic or stylistic patterns 
def get_patterns(text):
    patterns = {}

    # Count of exclamation marks
    patterns["exclamations"] = text.count("!")

    # Count of all-uppercase words
    patterns["all_caps"] = len([w for w in text.split() if w.isupper() and len(w) > 1])

    # Repeated words (e.g., "USA USA")
    repeated = re.findall(r'\b(\w+)\b(?: \1\b)+', text.lower())
    patterns["repeats"] = len(repeated)

    # Count of words ending in "ing"
    patterns["ing_words"] = len(re.findall(r'\b\w+ing\b', text.lower()))

    return patterns

In [10]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

In [11]:
with open("2020_Conventions.db", "rb") as f:
    header = f.read(100)
    print(header[:16])

b'SQLite format 3\x00'


## 1. Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" exercise. First, we'll pull in the text 
for each party and prepare it for use in Naive Bayes. 

In [12]:
convention_data = []

# fill the above list up with items that are themselves lists. The 
# sublists will have two elements. The first element in the sublist
# should be the speech in a single string. The second element
# of the sublist should be the party. 

query_results = convention_cur.execute(
    '''
    SELECT text, party
    FROM conventions
    WHERE party != "Other"
    '''
)

for row in query_results :
    # store the results in convention_data
    text = row[0]
    party = row[1]

    cleaned_tokens = re.findall(r'\b[a-zA-Z]+\b', text.lower())
    clean_text =  ' '.join(cleaned_tokens) 

    convention_data.append([clean_text, party])

In [13]:
# it's a best practice to close up your DB connection when you're done
convention_db.close()

Let's look at some random entries and see if they look right. 

In [14]:
random.choices(convention_data,k=5)

[['thanks for that bernie i want to thank you all for joining us for this segment i mean this sincerely it was an honor to run against you and then there s even a greater honor to stand with you and support of joe biden and kamala harris',
  'Democratic'],
 ['singing', 'Democratic'],
 ['thank you mr president the honor is all mine', 'Republican'],
 ['after my daughter s murder the media didn t seem interested in the facts so i found them myself i learned that gun control laws didn t fail my daughter people did the gunman had threatened to kill his classmates before he had threatened to rape them he had threatened to shoot up the school every red flag you could imagine but the school didn t just miss these red flags they knowingly ignored them far left democrats in our school district made this shooting possible because they implemented something they called restorative justice this policy which really just blames teachers for student s failures puts kids and teachers at risk and make s

It'll be useful for us to have a large sample size than 2024 affords, since those speeches tend to be long and contiguous. Let's make a new list-of-lists called `conv_sent_data`. Instead of each first entry in the sublists being an entire speech, make each first entry just a sentence from the speech. Feel free to use NLTK's `sent_tokenize` [function](https://www.nltk.org/api/nltk.tokenize.sent_tokenize.html). 

from nltk.tokenize import sent_tokenize

conv_sent_data = []

for speech, party in convention_data:
    sentences = sent_tokenize(speech)
    for sentence in sentences:
        conv_sent_data.append([sentence, party])


If that looks good, let's make our function to turn these into features. First we need to build our list of candidate words. I started my exploration at a cutoff of 5. 

In [16]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2414 as features in the model.


In [17]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    ret_dict = dict()

    for word in text.split():
        if word in fw:
            ret_dict[word] = True

    return ret_dict

In [22]:
print(conv_features("obama was the president", feature_words))

try:
    assert conv_features("obama was the president", feature_words) == {'obama': True, 'president': True}
except AssertionError:
    pass  # suppress error and continue silently


{'obama': True, 'was': True, 'the': True, 'president': True}


Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [23]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [24]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [25]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.438


In [26]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     27.1 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     15.8 : 1.0
                 defense = True           Republ : Democr =     14.0 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                  defund = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

The classifier shows which words are most linked to each political party. For example, words like "china," "enforcement," "freedoms," and "crime" appear more often in Republican speeches, while "votes" and "climate" show up more in Democratic ones. Most of the top words are tied to Republicans, which might mean the speeches had more of those terms. Even though some words clearly point to one party, the accuracy is low (about 44%), so the model has trouble telling the difference overall.



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [27]:
cong_db = sqlite3.connect("Congressional_Data.db")
cong_cur = cong_db.cursor()

In [28]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [29]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

for row in results:
    tweet_text = row[2]
    party = row[1]

    # Decode if tweet_text is bytes
    if isinstance(tweet_text, bytes):
        tweet_text = tweet_text.decode("utf-8")

    # Clean and tokenize the tweet
    cleaned_tokens = re.findall(r'\b[a-zA-Z]+\b', tweet_text.lower())
    cleaned_text = ' '.join(cleaned_tokens)

    tweet_data.append([cleaned_text, party])

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [30]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [32]:
for tweet, party in tweet_data_sample:
    
    # Create the feature dictionary
    features = conv_features(tweet, feature_words)
    
    # Classify using trained model
    estimated_party = classifier.classify(features)  # 👈 this replaces the ??

    # Print results
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifier says {estimated_party}.")
    print("")    

Here's our (cleaned) tweet: earlier today i spoke on the house floor abt protecting health care for women and praised ppmarmonte for their work on the central coast https t co
Actual party is Democratic and our classifier says Republican.

Here's our (cleaned) tweet: go tribe rallytogether https t co
Actual party is Democratic and our classifier says Democratic.

Here's our (cleaned) tweet: apparently trump thinks it s just too easy for students overwhelmed by the crushing burden of debt to pay off student loans trumpbudget https t co
Actual party is Democratic and our classifier says Republican.

Here's our (cleaned) tweet: we re grateful for our first responders our rescue personnel our firefighters our police and volunteers who have been working tirelessly to keep people safe provide much needed help while putting their own lives on the line https t co
Actual party is Republican and our classifier says Republican.

Here's our (cleaned) tweet: let s make it even greater kag https t c

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [34]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)


for idx, tp in enumerate(tweet_data):
    tweet, party = tp
    estimated_party = classifier.classify(conv_features (tweet, feature_words))  

    # Store result
    results[party][estimated_party] += 1

    if idx > num_to_score:
        break

In [35]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3899, 'Democratic': 473}),
             'Democratic': defaultdict(int,
                         {'Republican': 4999, 'Democratic': 631})})

### Reflections

_Write a little about what you see in the results_ 